In [30]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from scipy.stats import kurtosis, skew

# 1. 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')
dataY = np.loadtxt('dataset1/dataY.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 2. 数据预处理：合并 x, y, z 方向的数据
data = np.column_stack((dataX, dataY, dataZ))

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 3. 时域特征提取函数
def extract_features(data):
    features = []
    for group in data:
        feature = []
        feature.append(np.mean(group))         # 均值
        feature.append(np.std(group))          # 标准差
        feature.append(np.min(group))          # 最小值
        feature.append(np.max(group))          # 最大值
        feature.append(np.var(group))          # 方差
        feature.append(np.sqrt(np.mean(group**2)))  # 均方根
        feature.append(kurtosis(group))        # 峰度
        feature.append(skew(group))            # 偏度
        features.append(feature)
    return np.array(features)

# 提取 x, y, z 三个方向的时域特征
features_X = extract_features(dataX)
features_Y = extract_features(dataY)
features_Z = extract_features(dataZ)

# 将特征合并
data_features = np.column_stack((features_X, features_Y, features_Z))

# 4. 划分数据集
X_train, X_test, y_train_defect, y_test_defect, y_train_severity, y_test_severity = train_test_split(
    data_features, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

# 5. 创建 PyTorch 数据加载器
# 转换为 PyTorch 张量
train_tensor = torch.tensor(X_train, dtype=torch.float32)
test_tensor = torch.tensor(X_test, dtype=torch.float32)

train_labels_defect_tensor = torch.tensor(y_train_defect, dtype=torch.long)
test_labels_defect_tensor = torch.tensor(y_test_defect, dtype=torch.long)

train_labels_severity_tensor = torch.tensor(y_train_severity, dtype=torch.long)
test_labels_severity_tensor = torch.tensor(y_test_severity, dtype=torch.long)

# 创建 TensorDataset 和 DataLoader
train_dataset = TensorDataset(train_tensor, train_labels_defect_tensor, train_labels_severity_tensor)
test_dataset = TensorDataset(test_tensor, test_labels_defect_tensor, test_labels_severity_tensor)
batch_size = 1024  # 你可以在这里修改批大小
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("数据集划分完成")
print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")


数据集划分完成
训练集大小: 998
测试集大小: 250


In [31]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import precision_score, f1_score
import time

# 1. 定义注意力层
class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        # 定义可学习的注意力权重
        self.attention_weights = nn.Parameter(torch.randn(input_dim))

    def forward(self, x):
        # 计算注意力得分
        attention_scores = torch.matmul(x, self.attention_weights)  # [batch_size, input_dim] * [input_dim] -> [batch_size]
        attention_scores = torch.softmax(attention_scores, dim=-1)
        
        # 对输入进行加权
        weighted_output = torch.mul(x, attention_scores.unsqueeze(-1))  # 加权输入
        return weighted_output.sum(dim=1)  # 对特征进行加权求和

# 2. 改进模型以处理时域特征并添加注意力机制
class MLP_Model_With_Attention(nn.Module):
    def __init__(self, input_size=24):  # 输入维度根据时域特征的数量调整
        super(MLP_Model_With_Attention, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.attention = Attention(64)  # 添加注意力机制
        self.fc3 = nn.Linear(64, 3)  # 道路缺陷分类输出
        self.fc4 = nn.Linear(64, 4)  # 严重程度分类输出

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.attention(x)  # 应用注意力机制
        defect_output = self.fc3(x)  # 道路缺陷分类
        severity_output = self.fc4(x)  # 严重程度分类
        return defect_output, severity_output

# 3. 定义损失函数和优化器
model = MLP_Model_With_Attention(input_size=24)  # 这里的 input_size 是 8 * 3，每个方向提取8个特征
criterion = nn.CrossEntropyLoss()  # 分类任务的损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 计算模型参数数量
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"模型参数总数: {total_params}")

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # 将模型设置为训练模式
    start_time = time.time()  # 记录训练开始时间

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            # 打印输入的形状以检查
            print(f'Input shape: {inputs.shape}')
            
            optimizer.zero_grad()  # 清空梯度

            # 前向传播
            defect_preds, severity_preds = model(inputs)
            
            # 计算损失
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity

            # 反向传播和优化
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # 打印每个 epoch 的平均损失
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}')

    end_time = time.time()  # 记录训练结束时间
    print(f"训练时间: {end_time - start_time} 秒")

# 5. 测试模型并计算精确度、F1值
def test_model(model, test_loader):
    model.eval()  # 设置为评估模式
    correct_defect = 0
    correct_severity = 0
    total = 0
    all_defect_labels = []
    all_severity_labels = []
    all_defect_preds = []
    all_severity_preds = []

    with torch.no_grad():  # 禁用梯度计算
        for inputs, defect_labels, severity_labels in test_loader:
            defect_preds, severity_preds = model(inputs)

            # 预测
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            total += defect_labels.size(0)
            correct_defect += (predicted_defects == defect_labels).sum().item()
            correct_severity += (predicted_severities == severity_labels).sum().item()

            # 收集所有预测和真实值，用于计算精确度和F1值
            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算分类准确率
    accuracy_defect = 100 * correct_defect / total
    accuracy_severity = 100 * correct_severity / total

    # 计算精确度和F1值
    precision_defect = precision_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)
    precision_severity = precision_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)
    f1_defect = f1_score(all_defect_labels, all_defect_preds, average='weighted', zero_division=0)
    f1_severity = f1_score(all_severity_labels, all_severity_preds, average='weighted', zero_division=0)

    print(f'道路缺陷分类准确率: {accuracy_defect}%')
    print(f'严重程度分类准确率: {accuracy_severity}%')
    print(f'道路缺陷分类精确度: {precision_defect}')
    print(f'严重程度分类精确度: {precision_severity}')
    print(f'道路缺陷分类F1值: {f1_defect}')
    print(f'严重程度分类F1值: {f1_severity}')

# 6. 训练并测试模型
num_epochs = 100
train_model(model, train_loader, criterion, optimizer, num_epochs)
test_model(model, test_loader)


模型参数总数: 11975
Input shape: torch.Size([998, 24])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x998 and 64x3)

In [19]:
###超参数优化
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import kurtosis, skew

# 1. 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')
dataY = np.loadtxt('dataset1/dataY.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 2. 时域特征提取函数
def extract_time_features(data):
    features = []
    for group in data:
        feature = []
        feature.append(np.mean(group))         # 均值
        feature.append(np.std(group))          # 标准差
        feature.append(np.min(group))          # 最小值
        feature.append(np.max(group))          # 最大值
        feature.append(np.var(group))          # 方差
        feature.append(np.sqrt(np.mean(group**2)))  # 均方根
        feature.append(kurtosis(group))        # 峰度
        feature.append(skew(group))            # 偏度
        features.append(feature)
    return np.array(features)

# 3. 频域特征提取函数
def extract_frequency_features(data):
    features = []
    for group in data:
        fft_vals = np.fft.fft(group)
        amplitude_spectrum = np.abs(fft_vals)
        phase_spectrum = np.angle(fft_vals)

        # 计算频域特征
        feature = []
        feature.append(np.mean(amplitude_spectrum))        # 幅度谱均值
        feature.append(np.std(amplitude_spectrum))         # 幅度谱标准差
        feature.append(np.mean(phase_spectrum))            # 相位谱均值
        feature.append(np.std(phase_spectrum))             # 相位谱标准差
        feature.append(np.mean(amplitude_spectrum[:len(amplitude_spectrum)//2]))  # 低频部分均值
        feature.append(np.mean(amplitude_spectrum[len(amplitude_spectrum)//2:]))  # 高频部分均值
        features.append(feature)
    return np.array(features)

# 提取 x, y, z 三个方向的时域特征和频域特征
time_features_X = extract_time_features(dataX)
time_features_Y = extract_time_features(dataY)
time_features_Z = extract_time_features(dataZ)

freq_features_X = extract_frequency_features(dataX)
freq_features_Y = extract_frequency_features(dataY)
freq_features_Z = extract_frequency_features(dataZ)

# 将时域特征和频域特征合并
features_X = np.column_stack((time_features_X, freq_features_X))
features_Y = np.column_stack((time_features_Y, freq_features_Y))
features_Z = np.column_stack((time_features_Z, freq_features_Z))

# 将特征合并
data_features = np.column_stack((features_X, features_Y, features_Z))

# 4. 划分数据集
X_train, X_test, y_train_defect, y_test_defect, y_train_severity, y_test_severity = train_test_split(
    data_features, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

# 5. 定义模型
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_defect = nn.Linear(hidden_dim, 3)  # 道路缺陷分类输出 (假设是 3 类)
        self.fc_severity = nn.Linear(hidden_dim, 4)  # 严重程度分类输出 (假设是 4 类)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        defect_output = self.fc_defect(x)
        severity_output = self.fc_severity(x)
        return defect_output, severity_output

# 6. 定义训练和测试过程
def train_and_evaluate_model(learning_rate, batch_size, dropout_rate, hidden_dim):
    # 创建数据加载器
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                  torch.tensor(y_train_defect, dtype=torch.long),
                                  torch.tensor(y_train_severity, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                torch.tensor(y_test_defect, dtype=torch.long),
                                torch.tensor(y_test_severity, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型
    model = SimpleNN(input_dim=X_train.shape[1], hidden_dim=hidden_dim, dropout_rate=dropout_rate)
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    num_epochs = 20
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在验证集上评估模型
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []

    with torch.no_grad():
        for inputs, defect_labels, severity_labels in val_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算验证集上的准确率
    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)

    avg_accuracy = (accuracy_defect + accuracy_severity) / 2
    return -avg_accuracy

# 7. 使用 Optuna 进行超参数优化
import optuna

def objective(trial):
    learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])

    return train_and_evaluate_model(learning_rate, batch_size, dropout_rate, hidden_dim)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # 进行 50 次试验

# 输出最佳超参数
print("Best hyperparameters: ", study.best_params)

# 使用最佳超参数重新训练模型
best_params = study.best_params
train_and_evaluate_model(best_params['lr'], best_params['batch_size'], best_params['dropout_rate'], best_params['hidden_dim'])


[I 2024-09-10 21:53:40,759] Finished trial#0 resulted in value: -0.6599999999999999. Current best value is -0.6599999999999999 with parameters: {'lr': 3.311431091337471e-05, 'batch_size': 64, 'dropout_rate': 0.4502057046997324, 'hidden_dim': 64}.
[I 2024-09-10 21:53:42,100] Finished trial#1 resulted in value: -0.786. Current best value is -0.786 with parameters: {'lr': 0.0073762046111866915, 'batch_size': 16, 'dropout_rate': 0.11801314942267851, 'hidden_dim': 32}.
[I 2024-09-10 21:53:42,475] Finished trial#2 resulted in value: -0.8300000000000001. Current best value is -0.8300000000000001 with parameters: {'lr': 0.003151156696476796, 'batch_size': 128, 'dropout_rate': 0.3764513161927071, 'hidden_dim': 32}.
[I 2024-09-10 21:53:42,984] Finished trial#3 resulted in value: -0.774. Current best value is -0.8300000000000001 with parameters: {'lr': 0.003151156696476796, 'batch_size': 128, 'dropout_rate': 0.3764513161927071, 'hidden_dim': 32}.
[I 2024-09-10 21:53:44,012] Finished trial#4 resul

Best hyperparameters:  {'lr': 0.002335055708194196, 'batch_size': 128, 'dropout_rate': 0.13154845364657292, 'hidden_dim': 64}


-0.78

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import kurtosis, skew

# 1. 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')
dataY = np.loadtxt('dataset1/dataY.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 2. 时域特征提取函数
def extract_time_features(data):
    features = []
    for group in data:
        feature = []
        feature.append(np.mean(group))         # 均值
        feature.append(np.std(group))          # 标准差
        feature.append(np.min(group))          # 最小值
        feature.append(np.max(group))          # 最大值
        feature.append(np.var(group))          # 方差
        feature.append(np.sqrt(np.mean(group**2)))  # 均方根
        feature.append(kurtosis(group))        # 峰度
        feature.append(skew(group))            # 偏度
        features.append(feature)
    return np.array(features)

# 3. 频域特征提取函数
def extract_frequency_features(data):
    features = []
    for group in data:
        fft_vals = np.fft.fft(group)
        amplitude_spectrum = np.abs(fft_vals)
        phase_spectrum = np.angle(fft_vals)

        # 计算频域特征
        feature = []
        feature.append(np.mean(amplitude_spectrum))        # 幅度谱均值
        feature.append(np.std(amplitude_spectrum))         # 幅度谱标准差
        feature.append(np.mean(phase_spectrum))            # 相位谱均值
        feature.append(np.std(phase_spectrum))             # 相位谱标准差
        feature.append(np.mean(amplitude_spectrum[:len(amplitude_spectrum)//2]))  # 低频部分均值
        feature.append(np.mean(amplitude_spectrum[len(amplitude_spectrum)//2:]))  # 高频部分均值
        features.append(feature)
    return np.array(features)

# 提取 x, y, z 三个方向的时域特征和频域特征
time_features_X = extract_time_features(dataX)
time_features_Y = extract_time_features(dataY)
time_features_Z = extract_time_features(dataZ)

freq_features_X = extract_frequency_features(dataX)
freq_features_Y = extract_frequency_features(dataY)
freq_features_Z = extract_frequency_features(dataZ)

# 将时域特征和频域特征合并
features_X = np.column_stack((time_features_X, freq_features_X))
features_Y = np.column_stack((time_features_Y, freq_features_Y))
features_Z = np.column_stack((time_features_Z, freq_features_Z))

# 将特征合并
data_features = np.column_stack((features_X, features_Y, features_Z))

# 4. 划分数据集
X_train, X_test, y_train_defect, y_test_defect, y_train_severity, y_test_severity = train_test_split(
    data_features, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

# 5. 定义模型
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_defect = nn.Linear(hidden_dim, 3)  # 道路缺陷分类输出 (假设是 3 类)
        self.fc_severity = nn.Linear(hidden_dim, 4)  # 严重程度分类输出 (假设是 4 类)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        defect_output = self.fc_defect(x)
        severity_output = self.fc_severity(x)
        return defect_output, severity_output

# 6. 定义训练和测试过程
def train_and_evaluate_model(learning_rate, batch_size, dropout_rate, hidden_dim):
    # 创建数据加载器
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                  torch.tensor(y_train_defect, dtype=torch.long),
                                  torch.tensor(y_train_severity, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                torch.tensor(y_test_defect, dtype=torch.long),
                                torch.tensor(y_test_severity, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型
    model = SimpleNN(input_dim=X_train.shape[1], hidden_dim=hidden_dim, dropout_rate=dropout_rate)
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    num_epochs = 20
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在验证集上评估模型
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []

    with torch.no_grad():
        for inputs, defect_labels, severity_labels in val_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算验证集上的准确率
    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)

    avg_accuracy = (accuracy_defect + accuracy_severity) / 2
    return -avg_accuracy

# 7. 使用 Optuna 进行超参数优化
import optuna

def objective(trial):
    learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])

    return train_and_evaluate_model(learning_rate, batch_size, dropout_rate, hidden_dim)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # 进行 50 次试验

# 输出最佳超参数
print("Best hyperparameters: ", study.best_params)

# 使用最佳超参数重新训练模型
best_params = study.best_params
train_and_evaluate_model(best_params['lr'], best_params['batch_size'], best_params['dropout_rate'], best_params['hidden_dim'])


[I 2024-09-10 21:52:21,611] Finished trial#0 resulted in value: -0.812. Current best value is -0.812 with parameters: {'lr': 0.007172032511676547, 'batch_size': 32, 'dropout_rate': 0.37264165841506547, 'hidden_dim': 128}.
[I 2024-09-10 21:52:22,030] Finished trial#1 resulted in value: -0.636. Current best value is -0.812 with parameters: {'lr': 0.007172032511676547, 'batch_size': 32, 'dropout_rate': 0.37264165841506547, 'hidden_dim': 128}.
[I 2024-09-10 21:52:22,457] Finished trial#2 resulted in value: -0.724. Current best value is -0.812 with parameters: {'lr': 0.007172032511676547, 'batch_size': 32, 'dropout_rate': 0.37264165841506547, 'hidden_dim': 128}.
[I 2024-09-10 21:52:24,106] Finished trial#3 resulted in value: -0.794. Current best value is -0.812 with parameters: {'lr': 0.007172032511676547, 'batch_size': 32, 'dropout_rate': 0.37264165841506547, 'hidden_dim': 128}.
[I 2024-09-10 21:52:24,528] Finished trial#4 resulted in value: -0.8. Current best value is -0.812 with paramete

Best hyperparameters:  {'lr': 0.0020892701755892877, 'batch_size': 32, 'dropout_rate': 0.12769421124325098, 'hidden_dim': 128}


-0.8320000000000001

In [26]:
###只使用x，z轴两个通道的数据
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import kurtosis, skew

# 1. 加载数据
dataX = np.loadtxt('dataset1/dataX.txt')
dataZ = np.loadtxt('dataset1/dataZ.txt')
dataLabels = np.loadtxt('dataset1/dataLabel.txt')  # 假设每一行有两列

# 将标签分为两个部分：defect 标签和 severity 标签
labels_defect = dataLabels[:, 0].astype(int)  # 道路缺陷标签
labels_severity = dataLabels[:, 1].astype(int)  # 严重程度标签

# 2. 时域特征提取函数
def extract_time_features(data):
    features = []
    for group in data:
        feature = []
        feature.append(np.mean(group))         # 均值
        feature.append(np.std(group))          # 标准差
        feature.append(np.min(group))          # 最小值
        feature.append(np.max(group))          # 最大值
        feature.append(np.var(group))          # 方差
        feature.append(np.sqrt(np.mean(group**2)))  # 均方根
        feature.append(kurtosis(group))        # 峰度
        feature.append(skew(group))            # 偏度
        features.append(feature)
    return np.array(features)

# 3. 频域特征提取函数
def extract_frequency_features(data):
    features = []
    for group in data:
        fft_vals = np.fft.fft(group)
        amplitude_spectrum = np.abs(fft_vals)
        phase_spectrum = np.angle(fft_vals)

        # 计算频域特征
        feature = []
        feature.append(np.mean(amplitude_spectrum))        # 幅度谱均值
        feature.append(np.std(amplitude_spectrum))         # 幅度谱标准差
        feature.append(np.mean(phase_spectrum))            # 相位谱均值
        feature.append(np.std(phase_spectrum))             # 相位谱标准差
        feature.append(np.mean(amplitude_spectrum[:len(amplitude_spectrum)//2]))  # 低频部分均值
        feature.append(np.mean(amplitude_spectrum[len(amplitude_spectrum)//2:]))  # 高频部分均值
        features.append(feature)
    return np.array(features)

# 提取 x 和 z 轴的时域特征和频域特征
time_features_X = extract_time_features(dataX)
time_features_Z = extract_time_features(dataZ)

freq_features_X = extract_frequency_features(dataX)
freq_features_Z = extract_frequency_features(dataZ)

# 将时域特征和频域特征合并
features_X = np.column_stack((time_features_X, freq_features_X))
features_Z = np.column_stack((time_features_Z, freq_features_Z))

# 将 x 和 z 轴特征合并
data_features = np.column_stack((features_X, features_Z))

# 4. 划分数据集
X_train, X_test, y_train_defect, y_test_defect, y_train_severity, y_test_severity = train_test_split(
    data_features, labels_defect, labels_severity, test_size=0.2, stratify=labels_defect, random_state=42
)

# 5. 定义模型
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_defect = nn.Linear(hidden_dim, 3)  # 道路缺陷分类输出 (假设是 3 类)
        self.fc_severity = nn.Linear(hidden_dim, 4)  # 严重程度分类输出 (假设是 4 类)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        defect_output = self.fc_defect(x)
        severity_output = self.fc_severity(x)
        return defect_output, severity_output

# 6. 定义训练和测试过程
def train_and_evaluate_model(learning_rate, batch_size, dropout_rate, hidden_dim):
    # 创建数据加载器
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                  torch.tensor(y_train_defect, dtype=torch.long),
                                  torch.tensor(y_train_severity, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                torch.tensor(y_test_defect, dtype=torch.long),
                                torch.tensor(y_test_severity, dtype=torch.long))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 定义模型
    model = SimpleNN(input_dim=X_train.shape[1], hidden_dim=hidden_dim, dropout_rate=dropout_rate)
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    num_epochs = 200
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, defect_labels, severity_labels in train_loader:
            optimizer.zero_grad()
            defect_preds, severity_preds = model(inputs)
            loss_defect = criterion(defect_preds, defect_labels)
            loss_severity = criterion(severity_preds, severity_labels)
            loss = loss_defect + loss_severity
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # 在验证集上评估模型
    model.eval()
    all_defect_labels = []
    all_defect_preds = []
    all_severity_labels = []
    all_severity_preds = []

    with torch.no_grad():
        for inputs, defect_labels, severity_labels in val_loader:
            defect_preds, severity_preds = model(inputs)
            _, predicted_defects = torch.max(defect_preds, 1)
            _, predicted_severities = torch.max(severity_preds, 1)

            all_defect_labels.extend(defect_labels.cpu().numpy())
            all_defect_preds.extend(predicted_defects.cpu().numpy())
            all_severity_labels.extend(severity_labels.cpu().numpy())
            all_severity_preds.extend(predicted_severities.cpu().numpy())

    # 计算验证集上的准确率
    accuracy_defect = accuracy_score(all_defect_labels, all_defect_preds)
    accuracy_severity = accuracy_score(all_severity_labels, all_severity_preds)

    avg_accuracy = (accuracy_defect + accuracy_severity) / 2
    return -avg_accuracy

# 7. 使用 Optuna 进行超参数优化
import optuna

def objective(trial):
    learning_rate = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])

    return train_and_evaluate_model(learning_rate, batch_size, dropout_rate, hidden_dim)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # 进行 50 次试验

# 输出最佳超参数
print("Best hyperparameters: ", study.best_params)

# 使用最佳超参数重新训练模型
best_params = study.best_params
train_and_evaluate_model(best_params['lr'], best_params['batch_size'], best_params['dropout_rate'], best_params['hidden_dim'])


[I 2024-09-11 15:38:50,815] Finished trial#0 resulted in value: -0.8320000000000001. Current best value is -0.8320000000000001 with parameters: {'lr': 6.853180274516727e-05, 'batch_size': 32, 'dropout_rate': 0.35534506644482977, 'hidden_dim': 32}.
[I 2024-09-11 15:38:56,284] Finished trial#1 resulted in value: -0.852. Current best value is -0.852 with parameters: {'lr': 0.0006411904408964784, 'batch_size': 64, 'dropout_rate': 0.11866703883146675, 'hidden_dim': 64}.
[I 2024-09-11 15:39:06,088] Finished trial#2 resulted in value: -0.788. Current best value is -0.852 with parameters: {'lr': 0.0006411904408964784, 'batch_size': 64, 'dropout_rate': 0.11866703883146675, 'hidden_dim': 64}.
[I 2024-09-11 15:39:26,764] Finished trial#3 resulted in value: -0.804. Current best value is -0.852 with parameters: {'lr': 0.0006411904408964784, 'batch_size': 64, 'dropout_rate': 0.11866703883146675, 'hidden_dim': 64}.
[I 2024-09-11 15:39:42,528] Finished trial#4 resulted in value: -0.848. Current best v

KeyboardInterrupt: 